In [1]:
from Crypto.PublicKey import RSA
from Crypto.Util.number import *
f = open("pubkey.pem")
key = RSA.importKey(f.read())
c = open("flag.enc", mode="rb")
c = bytes_to_long(c.read())

def continued_fractions_expansion(numerator,denominator):#(e,N)
	result=[]
 
	divident = numerator % denominator
	quotient = numerator //denominator
	result.append(quotient)
 
	while divident != 0:
		numerator = numerator - quotient * denominator
 
		tmp = denominator
		denominator = numerator
		numerator = tmp
 
		divident = numerator % denominator
		quotient = numerator //denominator
		result.append(quotient)
 
	return result
 
def convergents(expansion):
	convergents=[(expansion[0], 1)]
	for i in range(1, len(expansion)):
		numerator = 1
		denominator = expansion[i]
		for j in range(i - 1, -1, -1):
			numerator += expansion[j] * denominator
			if j==0:
				break
			tmp = denominator
			denominator = numerator
			numerator = tmp
		convergents.append((numerator, denominator)) #(k,d)
	return convergents
 
def newtonSqrt(n):
	approx = n // 2
	better = (approx + n //approx) // 2
	while better != approx:
	    approx = better
	    better = (approx + n // approx) // 2
	return approx
 
def wiener_attack(cons, e, N):
	for cs in cons:
		k,d = cs
		if k == 0:
			continue
		phi_N = (e * d - 1) // k
		#x**2 - ((N - phi_N) + 1) * x + N = 0
		a = 1
		b = -((N - phi_N) + 1)
		c = N
		delta = b * b - 4 * a * c
		if delta <= 0:
			continue
		x1 = (newtonSqrt(delta) - b)//(2 * a)
		x2 = -(newtonSqrt(delta) + b)//(2 * a)
		if x1 * x2 == N:
			return [x1, x2, k, d]


In [2]:
n=key.n
e=key.e
expansion = continued_fractions_expansion(e, n)
cons = convergents(expansion)
p, q, k, d = wiener_attack(cons, e, n)
m = pow(c, d, n)

In [3]:
long_to_bytes(m)

b'HTB{b16_e_5m4ll_d_3qu4l5_w31n3r_4774ck}'